In [5]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import DBSCAN
from sklearn.metrics import confusion_matrix, classification_report
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesResampler

###############################################################################
# Parameters & Paths
###############################################################################
sample = "10"  # Process folder "10"
folder_path = sample            # Folder containing CSV files for sample 10
abnormal_txt_file = f"AbnormalTracks_{sample}.txt"  # Text file with abnormal track IDs
image_path = f"sample{sample}.jpg"  # Background image for visualization (optional)

# Create output directory if needed
output_dir = "trajectory_images"
os.makedirs(output_dir, exist_ok=True)

###############################################################################
# 1. Read Abnormal Track IDs and Determine Valid Files
###############################################################################
with open(abnormal_txt_file, "r") as file:
    abnormal_lines = [line.strip() for line in file.readlines() if line.strip()]
    abnormal_files_names = [f.split(".")[0] for f in abnormal_lines]

# List all CSV files available in the folder.
all_csv_files = set(os.listdir(folder_path))
print(len(all_csv_files), "CSV files found in folder:", folder_path)

# For each abnormal track ID, perform a prefix match to find a valid CSV file.
valid_abnormal_files = []
for name in abnormal_files_names:
    for csv_file_name in all_csv_files.copy():
        if csv_file_name.endswith(".csv") and name in csv_file_name:
            valid_abnormal_files.append(csv_file_name)
            all_csv_files.remove(csv_file_name)  # Remove matched file so remaining are normal
            break  # Use first match found

# Normal files are those CSVs that are not marked as abnormal.
valid_normal_files = [f for f in all_csv_files if f.endswith('.csv') and f not in valid_abnormal_files]

print("Valid abnormal files:", valid_abnormal_files)
print("Number of valid abnormal files:", len(valid_abnormal_files))
print("Valid normal files:", valid_normal_files)
print("Number of valid normal files:", len(valid_normal_files))

###############################################################################
# 2. Define Functions to Extract Trajectories & Preprocess
###############################################################################
def extract_trajectory(csv_path):
    """
    Reads a CSV file (with columns: frameNo, left, top, w, h) and returns the
    trajectory as an array of [frameNo, center_x, center_y] values.
    """
    df = pd.read_csv(csv_path, usecols=["frameNo", "left", "top", "w", "h"])
    df["center_x"] = df["left"] + df["w"] / 2
    df["center_y"] = df["top"] + df["h"] / 2
    return df[["frameNo", "center_x", "center_y"]].values

def normalize_trajectory(traj):
    """
    Normalize a trajectory by subtracting the starting point and scaling
    to make the features more comparable.
    """
    # Extract just the spatial coordinates
    spatial = traj[:, 1:3]
    
    # Center trajectory by subtracting the first point
    centered = spatial - spatial[0]
    
    # Calculate the max distance from origin for scaling
    max_dist = np.max(np.linalg.norm(centered, axis=1))
    if max_dist > 0:
        # Normalize to range [0, 1]
        normalized = centered / max_dist
    else:
        normalized = centered
    
    # Return with original frameNo
    result = np.column_stack((traj[:, 0], normalized))
    return result

def resample_trajectory(traj, n_points=50):
    """
    Resamples the trajectory to a fixed number of points using tslearn's TimeSeriesResampler.
    Also normalizes the trajectory for better comparison.
    """
    # First normalize the trajectory
    traj_norm = normalize_trajectory(traj)
    
    # Prepare for tslearn (requires 3D shape: [n_samples, n_timesteps, n_features])
    traj_3d = traj_norm.reshape(1, traj_norm.shape[0], traj_norm.shape[1])
    
    # Resample to fixed length
    resampler = TimeSeriesResampler(sz=n_points)
    traj_resampled = resampler.fit_transform(traj_3d)
    
    # Return the first (and only) sample
    return traj_resampled[0]

###############################################################################
# 3. Load Trajectories & Prepare Labels
###############################################################################
raw_trajectories = []  # Store original trajectories
processed_trajectories = []  # Store processed trajectories for clustering
labels = []        # 1 for abnormal, 0 for normal (based on file designation)
file_names = []    # Store file names for reference

# Process abnormal CSV files
for file_name in valid_abnormal_files:
    file_path = os.path.join(folder_path, file_name)
    traj = extract_trajectory(file_path)
    if len(traj) > 5:  # Ensure trajectory has enough points
        raw_trajectories.append(traj)
        processed_trajectories.append(resample_trajectory(traj, n_points=50))
        labels.append(1)  # Abnormal
        file_names.append(file_name)

# Process normal CSV files
for file_name in valid_normal_files:
    file_path = os.path.join(folder_path, file_name)
    traj = extract_trajectory(file_path)
    if len(traj) > 5:  # Ensure trajectory has enough points
        raw_trajectories.append(traj)
        processed_trajectories.append(resample_trajectory(traj, n_points=50))
        labels.append(0)  # Normal
        file_names.append(file_name)

print(f"Loaded {len(processed_trajectories)} trajectories from folder {folder_path}.")

###############################################################################
# 4. Compute DTW-based distance matrix for trajectories
###############################################################################
def compute_dtw_matrix(trajectories):
    """
    Compute a distance matrix for trajectories using Dynamic Time Warping (DTW).
    Only compares the spatial coordinates (x, y) and ignores the time component.
    """
    n = len(trajectories)
    dist_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            # Extract just the spatial dimensions for DTW
            seq1 = trajectories[i][:, 1:3]  # x, y coordinates
            seq2 = trajectories[j][:, 1:3]  # x, y coordinates
            
            # Compute DTW distance
            distance = dtw(seq1, seq2)
            
            # Fill the symmetric matrix
            dist_matrix[i, j] = distance
            dist_matrix[j, i] = distance
    
    return dist_matrix

# Compute distance matrix
dist_matrix = compute_dtw_matrix(processed_trajectories)

# Visualize the distance matrix (optional)
plt.figure(figsize=(10, 8))
plt.imshow(dist_matrix, cmap='viridis')
plt.colorbar(label='DTW Distance')
plt.title('DTW Distance Matrix between Trajectories')
plt.savefig(os.path.join(output_dir, f"dtw_distance_matrix_{sample}.png"))
plt.close()

###############################################################################
# 5. Apply DBSCAN for Clustering using the precomputed DTW distance matrix
###############################################################################
# The eps value needs to be tuned based on the distribution of distances
# Start with the median of the distances
eps_value = np.median(dist_matrix) * 0.5  # 50% of median distance as starting point
min_samples = max(3, int(len(processed_trajectories) * 0.05))  # At least 5% of total trajectories

print(f"Using eps={eps_value:.2f} and min_samples={min_samples}")

# Apply DBSCAN with precomputed distance matrix
dbscan = DBSCAN(
    eps=eps_value, 
    min_samples=min_samples,
    metric='precomputed'
)
clusters = dbscan.fit_predict(dist_matrix)

# Count clusters
unique_clusters = np.unique(clusters)
print(f"Found {len(unique_clusters)} clusters (including noise):")
for c in unique_clusters:
    count = np.sum(clusters == c)
    print(f"  Cluster {c}: {count} trajectories")

# Predicted labels: trajectories labeled as noise (cluster == -1) are abnormal
predicted_labels = [1 if cl == -1 else 0 for cl in clusters]

###############################################################################
# 6. Evaluate clustering results
###############################################################################
# Compare to ground truth labels
accuracy = np.mean(np.array(predicted_labels) == np.array(labels))
print(f"Accuracy: {accuracy:.4f}")

cm = confusion_matrix(labels, predicted_labels)
cr = classification_report(labels, predicted_labels)
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(cr)

# Create a DataFrame with results
results_df = pd.DataFrame({
    'filename': file_names,
    'true_label': labels,
    'cluster': clusters,
    'predicted_label': predicted_labels
})
results_df.to_csv(os.path.join(output_dir, f"clustering_results_{sample}.csv"), index=False)

###############################################################################
# 7. Visualize clusters
###############################################################################
# Color map for clusters (excluding noise)
cmap = plt.cm.get_cmap('tab10', max(2, len(unique_clusters)-1))

plt.figure(figsize=(12, 10))

# First plot the clusters (excluding noise points)
for i, traj in enumerate(processed_trajectories):
    if clusters[i] == -1:
        continue  # Skip noise points for now
    
    color = cmap(clusters[i] % 10)  # Cycle through 10 colors
    plt.plot(traj[:, 1], traj[:, 2], 
             color=color, alpha=0.5, 
             label=f"Cluster {clusters[i]}" if clusters[i] not in plt.gca().get_legend_handles_labels()[1] else "")

# Then plot the noise points on top
for i, traj in enumerate(processed_trajectories):
    if clusters[i] != -1:
        continue
    
    plt.plot(traj[:, 1], traj[:, 2], 
             'k--', alpha=0.7, linewidth=1,
             label="Anomaly" if "Anomaly" not in plt.gca().get_legend_handles_labels()[1] else "")

plt.title(f"Trajectory Clustering (Sample {sample})")
plt.xlabel("Normalized X")
plt.ylabel("Normalized Y")
plt.legend()
plt.savefig(os.path.join(output_dir, f"cluster_visualization_{sample}.png"))
plt.close()

###############################################################################
# 8. Visualize trajectories on original image (if available)
###############################################################################
def visualize_trajectories_on_image(image_path, trajectories, clusters, output_path):
    """
    Visualize multiple trajectories on the background image, colored by cluster.
    """
    # Try to read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Image {image_path} not found. Creating a blank canvas.")
        # Create a blank canvas
        max_x = max(np.max(traj[:, 1]) for traj in raw_trajectories)
        max_y = max(np.max(traj[:, 2]) for traj in raw_trajectories)
        image = np.ones((int(max_y) + 100, int(max_x) + 100, 3), dtype=np.uint8) * 255
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Color map for clusters
    cmap = plt.cm.get_cmap('tab10', max(2, len(np.unique(clusters))-1))
    
    # Plot each trajectory
    for i, traj in enumerate(trajectories):
        # Get points and convert to integer coordinates
        points = np.array([(int(p[1]), int(p[2])) for p in traj])
        
        # Determine color based on cluster
        if clusters[i] == -1:
            color = (255, 0, 0)  # Red for anomalies
            thickness = 2
        else:
            # Convert colormap value to BGR for OpenCV
            rgb = cmap(clusters[i] % 10)[:3]
            color = tuple(int(c * 255) for c in rgb)
            thickness = 1
            
        # Draw the trajectory line
        for j in range(len(points) - 1):
            cv2.line(image, points[j], points[j+1], color, thickness)
        
        # Mark start and end points
        cv2.circle(image, points[0], 5, color, -1)  # Start point
        cv2.circle(image, points[-1], 5, (0, 0, 0), -1)  # End point
    
    # Add a legend
    legend_y = 30
    cv2.putText(image, "Anomaly", (20, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (255, 0, 0), 2)
    
    for c in np.unique(clusters):
        if c == -1:
            continue
        legend_y += 30
        rgb = cmap(c % 10)[:3]
        color = tuple(int(c * 255) for c in rgb)
        cv2.putText(image, f"Cluster {c}", (20, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, color, 2)
    
    # Save the result
    plt.figure(figsize=(12, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

# Visualize trajectories on image if available
try:
    output_path = os.path.join(output_dir, f"trajectory_visualization_{sample}.png")
    visualize_trajectories_on_image(image_path, raw_trajectories, clusters, output_path)
    print(f"Visualization saved to {output_path}")
except Exception as e:
    print(f"Could not create visualization on image: {str(e)}")

###############################################################################
# 9. Parameter tuning helper function (optional)
###############################################################################
def tune_dbscan_parameters(dist_matrix, labels, eps_range=None, min_samples_range=None):
    """
    Tune DBSCAN parameters by trying different combinations of eps and min_samples.
    Returns a DataFrame with results for each parameter combination.
    """
    if eps_range is None:
        # Default: try values around the median distance
        median_dist = np.median(dist_matrix)
        eps_range = [median_dist * factor for factor in [0.3, 0.4, 0.5, 0.6, 0.7]]
    
    if min_samples_range is None:
        # Default: try percentages of the total number of samples
        n_samples = len(labels)
        min_samples_range = [max(2, int(n_samples * pct)) for pct in [0.02, 0.05, 0.1, 0.15]]
    
    results = []
    for eps in eps_range:
        for min_samples in min_samples_range:
            # Apply DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
            clusters = dbscan.fit_predict(dist_matrix)
            
            # Calculate metrics
            predicted = [1 if c == -1 else 0 for c in clusters]
            accuracy = np.mean(np.array(predicted) == np.array(labels))
            n_clusters = len(np.unique(clusters)) - (1 if -1 in clusters else 0)
            noise_ratio = np.sum(clusters == -1) / len(clusters)
            
            # Append results
            results.append({
                'eps': eps,
                'min_samples': min_samples,
                'n_clusters': n_clusters,
                'noise_ratio': noise_ratio,
                'accuracy': accuracy
            })
    
    # Convert to DataFrame
    return pd.DataFrame(results)

# Try different parameters if needed
# Uncomment to run parameter tuning
"""
tuning_results = tune_dbscan_parameters(dist_matrix, labels)
tuning_results.to_csv(os.path.join(output_dir, f"parameter_tuning_{sample}.csv"), index=False)

# Find best parameters by accuracy
best_params = tuning_results.loc[tuning_results['accuracy'].idxmax()]
print("Best parameters by accuracy:")
print(best_params)
"""

159 CSV files found in folder: 10
Valid abnormal files: ['768_.csv', '225_.csv', '332_340_350_388_389_402_410_467_481_515_542_.csv', '7_37_72_128_.csv', '436_.csv', '241_281_375_385_.csv', '22_36_54_142_152_164_192_257_258_276_282_301_334_416_455_487_508_588_597_645_745_811_853_882_916_.csv', '894_923_931_962_.csv', '341_501_.csv', '514_549_568_621_704_718_799_820_836_868_878_889_897_899_921_932_946_954_959_.csv', '232_249_285_304_344_418_.csv', '468_507_529_647_.csv', '823_.csv', '151_.csv', '421_.csv', '13_30_35_56_62_71_93_98_105_126_165_211_289_295_312_325_365_372_379_391_406_427_.csv', '23_75_119_160_231_245_278_308_321_682_708_769_843_864_.csv', '220_263_300_.csv', '835_.csv', '964_.csv', '493_544_.csv', '48_122_132_137_161_172_329_358_408_497_601_755_859_896_940_.csv', '20_47_73_141_229_322_359_420_447_506_577_602_607_699_767_.csv', '563_590_651_663_.csv', '370_.csv', '963_.csv', '620_.csv', '874_.csv', '430_523_550_555_571_.csv', '361_.csv', '382_423_.csv', '106_123_183_207_.cs

C:\Users\abhis\AppData\Local\Temp\ipykernel_41076\2369266870.py:228: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab10', max(2, len(unique_clusters)-1))
C:\Users\abhis\AppData\Local\Temp\ipykernel_41076\2369266870.py:277: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab10', max(2, len(np.unique(clusters))-1))


Visualization saved to trajectory_images\trajectory_visualization_10.png


'\ntuning_results = tune_dbscan_parameters(dist_matrix, labels)\ntuning_results.to_csv(os.path.join(output_dir, f"parameter_tuning_{sample}.csv"), index=False)\n\n# Find best parameters by accuracy\nbest_params = tuning_results.loc[tuning_results[\'accuracy\'].idxmax()]\nprint("Best parameters by accuracy:")\nprint(best_params)\n'

In [6]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import DBSCAN
from sklearn.metrics import confusion_matrix, classification_report
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesResampler
from scipy.interpolate import interp1d

###############################################################################
# Parameters & Paths
###############################################################################
sample = "10"  # Process folder "10"
folder_path = sample            # Folder containing CSV files for sample 10
abnormal_txt_file = f"AbnormalTracks_{sample}.txt"  # Text file with abnormal track IDs
image_path = f"sample{sample}.jpg"  # Background image for visualization (optional)

# Create output directory if needed
output_dir = "trajectory_images"
os.makedirs(output_dir, exist_ok=True)

###############################################################################
# 1. Read Abnormal Track IDs and Determine Valid Files
###############################################################################
with open(abnormal_txt_file, "r") as file:
    abnormal_lines = [line.strip() for line in file.readlines() if line.strip()]
    abnormal_files_names = [f.split(".")[0] for f in abnormal_lines]

# List all CSV files available in the folder.
all_csv_files = set(os.listdir(folder_path))
print(len(all_csv_files), "CSV files found in folder:", folder_path)

# For each abnormal track ID, perform a prefix match to find a valid CSV file.
valid_abnormal_files = []
for name in abnormal_files_names:
    for csv_file_name in all_csv_files.copy():
        if csv_file_name.endswith(".csv") and name in csv_file_name:
            valid_abnormal_files.append(csv_file_name)
            all_csv_files.remove(csv_file_name)  # Remove matched file so remaining are normal
            break  # Use first match found

# Normal files are those CSVs that are not marked as abnormal.
valid_normal_files = [f for f in all_csv_files if f.endswith('.csv') and f not in valid_abnormal_files]

print("Valid abnormal files:", valid_abnormal_files)
print("Number of valid abnormal files:", len(valid_abnormal_files))
print("Valid normal files:", valid_normal_files)
print("Number of valid normal files:", len(valid_normal_files))

###############################################################################
# 2. Define Functions to Extract Trajectories & Preprocess
###############################################################################
def extract_trajectory(csv_path):
    """
    Reads a CSV file (with columns: frameNo, left, top, w, h) and returns the
    trajectory as an array of [frameNo, center_x, center_y] values.
    """
    df = pd.read_csv(csv_path, usecols=["frameNo", "left", "top", "w", "h"])
    df["center_x"] = df["left"] + df["w"] / 2
    df["center_y"] = df["top"] + df["h"] / 2
    return df[["frameNo", "center_x", "center_y"]].values

def normalize_trajectory(traj):
    """
    Normalize a trajectory by scaling and centering.
    Instead of just using the first point, we use the entire trajectory's
    bounding box to normalize, preserving the overall shape.
    """
    # Extract just the spatial coordinates
    spatial = traj[:, 1:3].copy()
    
    # Find the bounding box
    min_coords = np.min(spatial, axis=0)
    max_coords = np.max(spatial, axis=0)
    
    # Center the trajectory to the middle of its bounding box
    center = (min_coords + max_coords) / 2
    centered = spatial - center
    
    # Scale to normalize
    scale = np.max(np.abs(centered)) if np.max(np.abs(centered)) > 0 else 1
    normalized = centered / scale
    
    # Return with original frameNo
    result = np.column_stack((traj[:, 0], normalized))
    return result

def resample_trajectory_uniform(traj, n_points=50):
    """
    Resamples the trajectory to a fixed number of points, ensuring even sampling
    across the entire path length, not just based on time/frame numbers.
    This prevents bias towards the beginning or end of trajectories.
    """
    # First normalize the trajectory
    traj_norm = normalize_trajectory(traj)
    
    # Extract components
    frames = traj_norm[:, 0]
    x_coords = traj_norm[:, 1]
    y_coords = traj_norm[:, 2]
    
    # Calculate cumulative path length (spatial only)
    points = np.column_stack((x_coords, y_coords))
    diffs = np.diff(points, axis=0)
    segment_lengths = np.sqrt(np.sum(diffs**2, axis=1))
    cum_length = np.insert(np.cumsum(segment_lengths), 0, 0)
    
    if cum_length[-1] == 0:  # Handle stationary trajectories
        return np.tile(traj_norm[0], (n_points, 1))
    
    # Create parameter t based on path length (not frame numbers)
    t_orig = cum_length / cum_length[-1]
    t_new = np.linspace(0, 1, n_points)
    
    # Interpolate each coordinate based on path length
    if len(t_orig) > 1:  # Only interpolate if we have multiple points
        x_interp = interp1d(t_orig, x_coords, kind='linear', bounds_error=False, fill_value="extrapolate")
        y_interp = interp1d(t_orig, y_coords, kind='linear', bounds_error=False, fill_value="extrapolate")
        frame_interp = interp1d(t_orig, frames, kind='linear', bounds_error=False, fill_value="extrapolate")
        
        # Create resampled trajectory
        new_x = x_interp(t_new)
        new_y = y_interp(t_new)
        new_frames = frame_interp(t_new)
        
        return np.column_stack((new_frames, new_x, new_y))
    else:
        # If only one point, return it repeated
        return np.tile(traj_norm[0], (n_points, 1))

###############################################################################
# 3. Calculate trajectory features to augment the analysis
###############################################################################
def compute_trajectory_features(traj):
    """
    Compute additional features from a trajectory to help identify abnormal patterns.
    Features include:
    - Total path length
    - Average speed
    - Maximum speed
    - Straightness (ratio of displacement to path length)
    - Number of direction changes beyond a threshold
    - Acceleration statistics
    """
    # Extract spatial coordinates
    spatial = traj[:, 1:3]
    frames = traj[:, 0]
    
    # Skip if too short
    if len(spatial) < 3:
        return [0, 0, 0, 0, 0, 0, 0]
    
    # Calculate displacements between consecutive points
    displacements = np.diff(spatial, axis=0)
    
    # Path length (sum of segment lengths)
    segment_lengths = np.sqrt(np.sum(displacements**2, axis=1))
    total_path_length = np.sum(segment_lengths)
    
    # Overall displacement (straight-line distance from start to end)
    overall_displacement = np.linalg.norm(spatial[-1] - spatial[0])
    
    # Straightness ratio (1.0 means perfectly straight line)
    straightness = overall_displacement / total_path_length if total_path_length > 0 else 0
    
    # Time intervals between frames
    time_intervals = np.diff(frames)
    
    # Speed calculation (units per frame)
    speeds = segment_lengths / time_intervals if len(time_intervals) > 0 else np.zeros_like(segment_lengths)
    avg_speed = np.mean(speeds) if len(speeds) > 0 else 0
    max_speed = np.max(speeds) if len(speeds) > 0 else 0
    
    # Direction changes
    if len(displacements) >= 2:
        # Normalize displacement vectors
        displacement_norms = np.sqrt(np.sum(displacements**2, axis=1))
        unit_displacements = np.divide(displacements, displacement_norms[:, np.newaxis], 
                                      where=displacement_norms[:, np.newaxis]>0)
        
        # Calculate dot products between consecutive displacement vectors
        dot_products = np.sum(unit_displacements[:-1] * unit_displacements[1:], axis=1)
        
        # Clamp to [-1, 1] to avoid numerical issues
        dot_products = np.clip(dot_products, -1, 1)
        
        # Convert to angles in degrees
        angles = np.arccos(dot_products) * 180 / np.pi
        
        # Count significant changes in direction (> 30 degrees)
        direction_changes = np.sum(angles > 30)
    else:
        direction_changes = 0
    
    # Acceleration calculation (change in speed)
    acceleration = np.diff(speeds) if len(speeds) > 1 else np.array([0])
    max_acceleration = np.max(np.abs(acceleration)) if len(acceleration) > 0 else 0
    avg_acceleration = np.mean(np.abs(acceleration)) if len(acceleration) > 0 else 0
    
    return [
        total_path_length,
        avg_speed,
        max_speed,
        straightness,
        direction_changes,
        max_acceleration,
        avg_acceleration
    ]

###############################################################################
# 4. Load Trajectories & Prepare Data
###############################################################################
raw_trajectories = []  # Store original trajectories
processed_trajectories = []  # Store processed trajectories for clustering
trajectory_features = []  # Store additional features
labels = []        # 1 for abnormal, 0 for normal (based on file designation)
file_names = []    # Store file names for reference

# Process abnormal CSV files
print("Processing abnormal files...")
for file_name in valid_abnormal_files:
    file_path = os.path.join(folder_path, file_name)
    traj = extract_trajectory(file_path)
    if len(traj) > 5:  # Ensure trajectory has enough points
        raw_trajectories.append(traj)
        resampled = resample_trajectory_uniform(traj, n_points=50)
        processed_trajectories.append(resampled)
        trajectory_features.append(compute_trajectory_features(traj))
        labels.append(1)  # Abnormal
        file_names.append(file_name)

# Process normal CSV files
print("Processing normal files...")
for file_name in valid_normal_files:
    file_path = os.path.join(folder_path, file_name)
    traj = extract_trajectory(file_path)
    if len(traj) > 5:  # Ensure trajectory has enough points
        raw_trajectories.append(traj)
        resampled = resample_trajectory_uniform(traj, n_points=50)
        processed_trajectories.append(resampled)
        trajectory_features.append(compute_trajectory_features(traj))
        labels.append(0)  # Normal
        file_names.append(file_name)

print(f"Loaded {len(processed_trajectories)} trajectories from folder {folder_path}.")

# Convert trajectory features to array for analysis
trajectory_features = np.array(trajectory_features)
feature_names = [
    'path_length', 'avg_speed', 'max_speed', 'straightness', 
    'direction_changes', 'max_acceleration', 'avg_acceleration'
]

# Visualize feature distributions by label
plt.figure(figsize=(16, 12))
for i, feature in enumerate(feature_names):
    plt.subplot(3, 3, i+1)
    for label, color, name in zip([0, 1], ['blue', 'red'], ['Normal', 'Abnormal']):
        idx = np.where(np.array(labels) == label)[0]
        if len(idx) > 0:
            plt.hist(trajectory_features[idx, i], alpha=0.5, color=color, label=name, bins=10)
    plt.xlabel(feature)
    plt.title(f'Distribution of {feature}')
    if i == 0:
        plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"feature_distributions_{sample}.png"))
plt.close()

###############################################################################
# 5. Compute DTW-based distance matrix for trajectories
###############################################################################
def compute_dtw_matrix(trajectories, feature_weights=None):
    """
    Compute a distance matrix for trajectories using Dynamic Time Warping (DTW).
    Only compares the spatial coordinates (x, y) and ignores the time component.
    Optionally incorporates additional feature distances.
    """
    n = len(trajectories)
    dist_matrix = np.zeros((n, n))
    
    # Normalize features if provided
    if feature_weights is not None:
        # Standardize features
        feature_means = np.mean(trajectory_features, axis=0)
        feature_stds = np.std(trajectory_features, axis=0)
        feature_stds[feature_stds == 0] = 1  # Avoid division by zero
        norm_features = (trajectory_features - feature_means) / feature_stds
    
    for i in range(n):
        for j in range(i, n):
            # Extract just the spatial dimensions for DTW
            seq1 = trajectories[i][:, 1:3]  # x, y coordinates
            seq2 = trajectories[j][:, 1:3]  # x, y coordinates
            
            # Compute DTW distance
            dtw_distance = dtw(seq1, seq2)
            
            # Optionally add weighted feature distance
            distance = dtw_distance
            if feature_weights is not None:
                # Compute Euclidean distance between normalized features
                feature_dist = np.linalg.norm(
                    (norm_features[i] - norm_features[j]) * feature_weights
                )
                # Combine DTW and feature distances
                distance = dtw_distance + feature_dist
            
            # Fill the symmetric matrix
            dist_matrix[i, j] = distance
            dist_matrix[j, i] = distance
    
    return dist_matrix

# You can experiment with different feature weights to emphasize certain aspects
# Weights are in the same order as feature_names list
feature_weights = np.array([0.2, 0.3, 0.3, 0.5, 0.4, 0.3, 0.3])

# Compute distance matrix with DTW + feature weights
print("Computing distance matrix...")
dist_matrix = compute_dtw_matrix(processed_trajectories, feature_weights=feature_weights)

# Visualize the distance matrix
plt.figure(figsize=(10, 8))
plt.imshow(dist_matrix, cmap='viridis')
plt.colorbar(label='Distance')
plt.title('Distance Matrix between Trajectories')
plt.savefig(os.path.join(output_dir, f"distance_matrix_{sample}.png"))
plt.close()

###############################################################################
# 6. Apply DBSCAN for Clustering using the precomputed distance matrix
###############################################################################
# The eps value needs to be tuned based on the distribution of distances
# Start with a percentile of the distances to get a better estimate
distances_flattened = dist_matrix[np.triu_indices(len(dist_matrix), k=1)]
eps_percentiles = np.percentile(distances_flattened, [10, 25, 50, 75])
print(f"Distance percentiles: {eps_percentiles}")

# Select eps as the 25th percentile (can be adjusted based on data)
eps_value = eps_percentiles[1]  # 25th percentile
min_samples = max(3, int(len(processed_trajectories) * 0.05))  # At least 5% of total trajectories

print(f"Using eps={eps_value:.2f} and min_samples={min_samples}")

# Apply DBSCAN with precomputed distance matrix
dbscan = DBSCAN(
    eps=eps_value, 
    min_samples=min_samples,
    metric='precomputed'
)
clusters = dbscan.fit_predict(dist_matrix)

# Count clusters
unique_clusters = np.unique(clusters)
print(f"Found {len(unique_clusters)} clusters (including noise):")
for c in unique_clusters:
    count = np.sum(clusters == c)
    print(f"  Cluster {c}: {count} trajectories")

# Predicted labels: trajectories labeled as noise (cluster == -1) are abnormal
predicted_labels = [1 if cl == -1 else 0 for cl in clusters]

###############################################################################
# 7. Evaluate clustering results
###############################################################################
# Compare to ground truth labels
accuracy = np.mean(np.array(predicted_labels) == np.array(labels))
print(f"Accuracy: {accuracy:.4f}")

cm = confusion_matrix(labels, predicted_labels)
cr = classification_report(labels, predicted_labels)
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(cr)

# Analyze misclassifications in detail
misclassified_indices = np.where(np.array(predicted_labels) != np.array(labels))[0]
if len(misclassified_indices) > 0:
    print(f"Found {len(misclassified_indices)} misclassified trajectories:")
    for idx in misclassified_indices:
        print(f"  File: {file_names[idx]}, True: {labels[idx]}, Predicted: {predicted_labels[idx]}")
        
        # Analyze features of misclassified trajectories
        features = trajectory_features[idx]
        print(f"  Features: {dict(zip(feature_names, features))}")

# Create a DataFrame with results
results_df = pd.DataFrame({
    'filename': file_names,
    'true_label': labels,
    'cluster': clusters,
    'predicted_label': predicted_labels
})

# Add features to the results DataFrame
for i, name in enumerate(feature_names):
    results_df[name] = trajectory_features[:, i]

results_df.to_csv(os.path.join(output_dir, f"clustering_results_{sample}.csv"), index=False)

###############################################################################
# 8. Visualize clusters
###############################################################################
# Color map for clusters (excluding noise)
cmap = plt.cm.get_cmap('tab10', max(2, len(unique_clusters)-1))

plt.figure(figsize=(12, 10))

# First plot the clusters (excluding noise points)
for i, traj in enumerate(processed_trajectories):
    if clusters[i] == -1:
        continue  # Skip noise points for now
    
    color = cmap(clusters[i] % 10)  # Cycle through 10 colors
    plt.plot(traj[:, 1], traj[:, 2], 
             color=color, alpha=0.5, 
             label=f"Cluster {clusters[i]}" if clusters[i] not in plt.gca().get_legend_handles_labels()[1] else "")

# Then plot the noise points on top
for i, traj in enumerate(processed_trajectories):
    if clusters[i] != -1:
        continue
    
    plt.plot(traj[:, 1], traj[:, 2], 
             'r--', alpha=0.7, linewidth=1,
             label="Anomaly" if "Anomaly" not in plt.gca().get_legend_handles_labels()[1] else "")

plt.title(f"Trajectory Clustering (Sample {sample})")
plt.xlabel("Normalized X")
plt.ylabel("Normalized Y")
plt.legend()
plt.savefig(os.path.join(output_dir, f"cluster_visualization_{sample}.png"))
plt.close()

###############################################################################
# 9. Visualize trajectories on original image (if available)
###############################################################################
def visualize_trajectories_on_image(image_path, trajectories, clusters, labels, output_path):
    """
    Visualize multiple trajectories on the background image, colored by cluster.
    Mark misclassifications with special symbols.
    """
    # Try to read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Image {image_path} not found. Creating a blank canvas.")
        # Create a blank canvas
        max_x = max(np.max(traj[:, 1]) for traj in raw_trajectories)
        max_y = max(np.max(traj[:, 2]) for traj in raw_trajectories)
        image = np.ones((int(max_y) + 100, int(max_x) + 100, 3), dtype=np.uint8) * 255
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Color map for clusters
    cmap = plt.cm.get_cmap('tab10', max(2, len(np.unique(clusters))-1))
    
    # Convert clusters to predicted labels
    predicted = np.array([1 if c == -1 else 0 for c in clusters])
    
    # Plot each trajectory
    for i, traj in enumerate(trajectories):
        # Get points and convert to integer coordinates
        points = np.array([(int(p[1]), int(p[2])) for p in traj])
        
        # Skip if no points
        if len(points) == 0:
            continue
            
        # Determine color based on cluster
        if clusters[i] == -1:
            color = (255, 0, 0)  # Red for anomalies
            thickness = 2
        else:
            # Convert colormap value to BGR for OpenCV
            rgb = cmap(clusters[i] % 10)[:3]
            color = tuple(int(c * 255) for c in rgb)
            thickness = 1
            
        # Draw the trajectory line
        for j in range(len(points) - 1):
            cv2.line(image, points[j], points[j+1], color, thickness)
        
        # Mark start points
        cv2.circle(image, points[0], 5, color, -1)
        
        # Mark end points and misclassifications
        if predicted[i] != labels[i]:
            # Mark misclassifications with an X
            end_pt = points[-1]
            size = 8
            cv2.line(image, (end_pt[0]-size, end_pt[1]-size), 
                     (end_pt[0]+size, end_pt[1]+size), (0, 255, 255), 2)
            cv2.line(image, (end_pt[0]-size, end_pt[1]+size), 
                     (end_pt[0]+size, end_pt[1]-size), (0, 255, 255), 2)
        else:
            # Normal endpoint marker
            cv2.circle(image, points[-1], 5, (0, 0, 0), -1)
    
    # Add a legend
    legend_y = 30
    cv2.putText(image, "Anomaly", (20, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (255, 0, 0), 2)
    legend_y += 30
    cv2.putText(image, "Misclassified", (20, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 255, 255), 2)
    
    for c in np.unique(clusters):
        if c == -1:
            continue
        legend_y += 30
        rgb = cmap(c % 10)[:3]
        color = tuple(int(c * 255) for c in rgb)
        cv2.putText(image, f"Cluster {c}", (20, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, color, 2)
    
    # Save the result
    plt.figure(figsize=(12, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

# Visualize trajectories on image if available
try:
    output_path = os.path.join(output_dir, f"trajectory_visualization_{sample}.png")
    visualize_trajectories_on_image(image_path, raw_trajectories, clusters, labels, output_path)
    print(f"Visualization saved to {output_path}")
except Exception as e:
    print(f"Could not create visualization on image: {str(e)}")

###############################################################################
# 10. Parameter tuning helper function 
###############################################################################
def tune_dbscan_parameters(dist_matrix, labels, eps_range=None, min_samples_range=None):
    """
    Tune DBSCAN parameters by trying different combinations of eps and min_samples.
    Returns a DataFrame with results for each parameter combination.
    """
    if eps_range is None:
        # Use percentiles of the distance distribution
        distances = dist_matrix[np.triu_indices(len(dist_matrix), k=1)]
        eps_range = np.percentile(distances, [10, 15, 20, 25, 30, 35, 40, 50])
    
    if min_samples_range is None:
        # Try different percentages of the total number of samples
        n_samples = len(labels)
        min_samples_range = [max(2, int(n_samples * pct)) for pct in [0.01, 0.02, 0.05, 0.1, 0.15]]
    
    results = []
    for eps in eps_range:
        for min_samples in min_samples_range:
            # Apply DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
            clusters = dbscan.fit_predict(dist_matrix)
            
            # Calculate metrics
            predicted = [1 if c == -1 else 0 for c in clusters]
            accuracy = np.mean(np.array(predicted) == np.array(labels))
            n_clusters = len(np.unique(clusters)) - (1 if -1 in clusters else 0)
            noise_ratio = np.sum(clusters == -1) / len(clusters)
            
            # Calculate precision, recall, and F1 score for abnormal class
            true_positives = np.sum((np.array(labels) == 1) & (np.array(predicted) == 1))
            false_positives = np.sum((np.array(labels) == 0) & (np.array(predicted) == 1))
            false_negatives = np.sum((np.array(labels) == 1) & (np.array(predicted) == 0))
            
            precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
            recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            # Append results
            results.append({
                'eps': eps,
                'min_samples': min_samples,
                'n_clusters': n_clusters,
                'noise_ratio': noise_ratio,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1
            })
    
    # Convert to DataFrame
    return pd.DataFrame(results)

# Run parameter tuning to find the best parameters
print("Running parameter tuning...")
tuning_results = tune_dbscan_parameters(dist_matrix, labels)
tuning_results.to_csv(os.path.join(output_dir, f"parameter_tuning_{sample}.csv"), index=False)

# Find best parameters by F1 score (better for imbalanced classes)
best_params = tuning_results.loc[tuning_results['f1_score'].idxmax()]
print("Best parameters by F1 score:")
print(best_params)

# Re-run with best parameters
best_eps = best_params['eps']
best_min_samples = int(best_params['min_samples'])
print(f"Re-running with best parameters: eps={best_eps:.4f}, min_samples={best_min_samples}")

# Apply DBSCAN with best parameters
best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples, metric='precomputed')
best_clusters = best_dbscan.fit_predict(dist_matrix)
best_predicted = [1 if c == -1 else 0 for c in best_clusters]

# Evaluate final results
final_accuracy = np.mean(np.array(best_predicted) == np.array(labels))
final_cm = confusion_matrix(labels, best_predicted)
final_cr = classification_report(labels, best_predicted)

print(f"Final accuracy: {final_accuracy:.4f}")
print("Final confusion matrix:")
print(final_cm)
print("Final classification report:")
print(final_cr)

# Save final results
final_results = pd.DataFrame({
    'filename': file_names,
    'true_label': labels,
    'cluster': best_clusters,
    'predicted_label': best_predicted
})

# Add features to the results
for i, name in enumerate(feature_names):
    final_results[name] = trajectory_features[:, i]

final_results.to_csv(os.path.join(output_dir, f"final_results_{sample}.csv"), index=False)

# Final visualization with best parameters
try:
    output_path = os.path.join(output_dir, f"final_visualization_{sample}.png")
    visualize_trajectories_on_image(image_path, raw_trajectories, best_clusters, labels, output_path)
    print(f"Final visualization saved to {output_path}")
except Exception as e:
    print(f"Could not create final visualization: {str(e)}")

159 CSV files found in folder: 10
Valid abnormal files: ['768_.csv', '225_.csv', '332_340_350_388_389_402_410_467_481_515_542_.csv', '7_37_72_128_.csv', '436_.csv', '241_281_375_385_.csv', '22_36_54_142_152_164_192_257_258_276_282_301_334_416_455_487_508_588_597_645_745_811_853_882_916_.csv', '894_923_931_962_.csv', '341_501_.csv', '514_549_568_621_704_718_799_820_836_868_878_889_897_899_921_932_946_954_959_.csv', '232_249_285_304_344_418_.csv', '468_507_529_647_.csv', '823_.csv', '151_.csv', '421_.csv', '13_30_35_56_62_71_93_98_105_126_165_211_289_295_312_325_365_372_379_391_406_427_.csv', '23_75_119_160_231_245_278_308_321_682_708_769_843_864_.csv', '220_263_300_.csv', '835_.csv', '964_.csv', '493_544_.csv', '48_122_132_137_161_172_329_358_408_497_601_755_859_896_940_.csv', '20_47_73_141_229_322_359_420_447_506_577_602_607_699_767_.csv', '563_590_651_663_.csv', '370_.csv', '963_.csv', '620_.csv', '874_.csv', '430_523_550_555_571_.csv', '361_.csv', '382_423_.csv', '106_123_183_207_.cs

c:\Users\abhis\anaconda3\envs\test\Lib\site-packages\scipy\interpolate\_interpolate.py:712: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\abhis\anaconda3\envs\test\Lib\site-packages\scipy\interpolate\_interpolate.py:712: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\abhis\anaconda3\envs\test\Lib\site-packages\scipy\interpolate\_interpolate.py:715: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
c:\Users\abhis\anaconda3\envs\test\Lib\site-packages\scipy\interpolate\_interpolate.py:712: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\abhis\anaconda3\envs\test\Lib\site-packages\scipy\interpolate\_interpolate.py:712: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\abhis\anaconda3\envs\tes

Loaded 130 trajectories from folder 10.
Computing distance matrix...
Distance percentiles: [nan nan nan nan]
Using eps=nan and min_samples=6


InvalidParameterError: The 'eps' parameter of DBSCAN must be a float in the range (0.0, inf). Got np.float64(nan) instead.